In [1]:
import torch
import yaml
from config import BitformerConfig
from model_zoo import VisionBitformerForImageClassification
from trainer import get_trainer
from data_zoo import get_mnist

In [2]:
yaml_path = './yamls/mnist.yaml'
with open(yaml_path, 'r') as file:
    args = yaml.safe_load(file)

In [3]:
args

{'general_config': {'model_type': 'VisionBitformerForImageClassification',
  'num_labels': 10},
 'model_config': {'vocab_size': 32000,
  'hidden_size': 28,
  'intermediate_size': 512,
  'num_hidden_layers': 1,
  'num_attention_heads': 2,
  'num_key_value_heads': 2,
  'hidden_act': 'silu',
  'max_position_embeddings': 4096,
  'initializer_range': 0.02,
  'rms_norm_eps': 1e-05,
  'use_cache': False,
  'pad_token_id': None,
  'bos_token_id': 1,
  'eos_token_id': 2,
  'tie_word_embeddings': False,
  'rope_theta': 1000000.0,
  'sliding_window': 4096,
  'attention_dropout': 0.0,
  'num_experts_per_tok': 2,
  'num_local_experts': 4,
  'output_router_logits': True,
  'router_aux_loss_coef': 0.001,
  'is_causal': False,
  'moe': True,
  'bitnet': False},
 'training_args': {'output_dir': './results',
  'logging_dir': './logs',
  'report_to': None,
  'evaluation_strategy': 'epoch',
  'per_device_train_batch_size': 64,
  'per_device_eval_batch_size': 64,
  'gradient_accumulation_steps': 1,
  'lear

In [4]:
cfg = BitformerConfig(**args['model_config'], num_labels=args['general_config']['num_labels'])
model = VisionBitformerForImageClassification(config=cfg)
model.num_parameters() / 1e6

1.071644

In [5]:
from torchvision import transforms, datasets


def data_collator(batch):
    pixel_values = torch.stack([item[0].squeeze(0).reshape(28, 28) for item in batch])
    labels = torch.tensor([item[1] for item in batch])
    return {'inputs_embeds': pixel_values, 'labels': labels}

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST('./mnist_data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('./mnist_data', train=False, download=True, transform=transform)

In [6]:
trainer = get_trainer(model, train_dataset, test_dataset, data_collator, **args['training_args'])

In [7]:
trainer.train()
trainer.evaluate()

  0%|          | 0/9380 [00:00<?, ?it/s]

Could not estimate the number of tokens of the input, floating-point operations will not be computed


{'loss': 1.8517, 'grad_norm': 2.1099369525909424, 'learning_rate': 0.0004977113991447017, 'epoch': 0.53}


  0%|          | 0/157 [00:00<?, ?it/s]

Confusion Matrix:
[[ 765    0    7    0    6   25   37    1  134    5]
 [   0 1115    5    2    0    4    6    0    3    0]
 [  20    0  675  174   27   54   64    4   14    0]
 [   4   17   97  745   10   42    4   51   17   23]
 [  41    0   13   16  673    7   22   13   28  169]
 [  41   10   87  173   25  360   29   20  135   12]
 [  51   14   31    0    6   13  832    0   11    0]
 [   0   14   14   53   10    1    0  838    3   95]
 [  68   17    4   21   22   27   22    2  766   25]
 [  12    8    1   19   73    5    0  134   22  735]]
{'eval_loss': 0.7526538968086243, 'eval_f1': 0.7456515832870662, 'eval_precision': 0.7501509712491882, 'eval_recall': 0.7504, 'eval_accuracy': 0.7504, 'eval_runtime': 2.1495, 'eval_samples_per_second': 4652.276, 'eval_steps_per_second': 73.041, 'epoch': 1.0}
{'loss': 0.9053, 'grad_norm': 1.7508418560028076, 'learning_rate': 0.0004884857252366847, 'epoch': 1.07}
{'loss': 0.6585, 'grad_norm': 2.313500165939331, 'learning_rate': 0.0004724434323532821

...

Checkpoint destination directory ./results\checkpoint-9380 already exists and is non-empty. Saving will proceed but saved results may be invalid.


Confusion Matrix:
[[ 945    0    2    1    0    5   15    0    9    3]
 [   0 1105    6    2    0    4    6    0   11    1]
 [   6    3  896   61    5   17   22    9   10    3]
 [   0    3   40  925    0   16    0   12   10    4]
 [   0    2    4    1  907    1    3    9    6   49]
 [   2    4   11   47    3  786    5    3   24    7]
 [   7    3   10    1   10    9  910    1    7    0]
 [   0   14   17   13   13    3    0  925    1   42]
 [  24    9    9   22    7   22   10    3  861    7]
 [   5    8    6   12   36    5    0   30   15  892]]
{'eval_loss': 0.2666623294353485, 'eval_f1': 0.9151400967608606, 'eval_precision': 0.9154825760974205, 'eval_recall': 0.9152, 'eval_accuracy': 0.9152, 'eval_runtime': 2.1742, 'eval_samples_per_second': 4599.459, 'eval_steps_per_second': 72.212, 'epoch': 10.0}
{'train_runtime': 194.9569, 'train_samples_per_second': 3077.603, 'train_steps_per_second': 48.113, 'train_loss': 0.45974236998730883, 'epoch': 10.0}


  0%|          | 0/157 [00:00<?, ?it/s]

Confusion Matrix:
[[ 942    0    2    1    0    6   16    0   10    3]
 [   0 1106    6    2    0    4    6    0   10    1]
 [   6    3  901   61    5   16   19    8   10    3]
 [   0    3   39  930    0   16    0   12    6    4]
 [   0    2    4    1  907    1    3   10    6   48]
 [   2    4   10   47    3  788    5    3   23    7]
 [   7    3   10    1   10    9  910    1    7    0]
 [   0   14   17   12   13    3    0  928    1   40]
 [  22    8    9   23    7   26   10    3  859    7]
 [   4    9    6   16   35    5    0   31   14  889]]


{'eval_loss': 0.2673957645893097,
 'eval_f1': 0.9159655975632608,
 'eval_precision': 0.9163724161435439,
 'eval_recall': 0.916,
 'eval_accuracy': 0.916,
 'eval_runtime': 2.2432,
 'eval_samples_per_second': 4457.904,
 'eval_steps_per_second': 69.989,
 'epoch': 10.0}

In [7]:
trainer.train()

  0%|          | 0/9380 [00:00<?, ?it/s]

Could not estimate the number of tokens of the input, floating-point operations will not be computed


{'loss': 1.5768, 'grad_norm': 6.464507102966309, 'learning_rate': 0.0004977113991447017, 'epoch': 0.53}


  0%|          | 0/157 [00:00<?, ?it/s]

Confusion Matrix:
[[ 877    0    6    1   24    7   22    1   41    1]
 [   0 1027   16   13    3   46    6    7   15    2]
 [  50    5  648   57   40   87   65    3   69    8]
 [   2    4  105  701   11   52    2   18   48   67]
 [  43    4   18    2  659    9    4    8  163   72]
 [  33   11   44   54   37  581   14    6   87   25]
 [  18    2   15    2    3    5  846    0   67    0]
 [   0   12    2   12   45   20    0  696    5  236]
 [  55    1   12   10   21   22   10    3  823   17]
 [   8    4    1    9  170    3    1   15   48  750]]
{'eval_loss': 0.7616570591926575, 'eval_f1': 0.7619962587705958, 'eval_precision': 0.7746075940242815, 'eval_recall': 0.7608, 'eval_accuracy': 0.7608, 'eval_runtime': 3.6155, 'eval_samples_per_second': 2765.841, 'eval_steps_per_second': 43.424, 'epoch': 1.0}
{'loss': 0.8529, 'grad_norm': 10.953009605407715, 'learning_rate': 0.0004884857252366847, 'epoch': 1.07}
{'loss': 0.6767, 'grad_norm': 20.91096305847168, 'learning_rate': 0.0004724434323532821

...

  0%|          | 0/157 [00:00<?, ?it/s]

Confusion Matrix:
[[ 925    0    4    0    9    4    8    2   21    7]
 [   0 1116    2    1    1    4    5    0    6    0]
 [   8   11  944   23    6   19    5   10    5    1]
 [   1    9   16  943    0   24    0   10    5    2]
 [   3    3    7    3  875   13   11    6   19   42]
 [   5    2    8   27    2  824    5    1   12    6]
 [   4    2    2    4   10    9  920    0    7    0]
 [   0   15   14   13   21    2    0  900    6   57]
 [  12    4    4   17    2   22    9    0  900    4]
 [   6    4    1   14   30   12    0   12   16  914]]
{'eval_loss': 0.242593914270401, 'eval_f1': 0.9260794668701173, 'eval_precision': 0.9267383853604548, 'eval_recall': 0.9261, 'eval_accuracy': 0.9261, 'eval_runtime': 3.6315, 'eval_samples_per_second': 2753.698, 'eval_steps_per_second': 43.233, 'epoch': 9.0}
{'loss': 0.2442, 'grad_norm': 4.718122959136963, 'learning_rate': 1.1011964332097113e-05, 'epoch': 9.06}
{'loss': 0.2297, 'grad_norm': 4.965157985687256, 'learning_rate': 2.065770110498438e-06,

  0%|          | 0/157 [00:00<?, ?it/s]

Confusion Matrix:
[[ 944    0    5    0    6    3    7    2   12    1]
 [   0 1119    3    2    0    1    4    0    6    0]
 [   8    9  968   20    2   11    4    8    2    0]
 [   2    8   21  949    0   12    1   10    4    3]
 [   2    5   11    2  897   10   12    7   10   26]
 [   3    3   15   35    2  808    8    1   12    5]
 [   5    2    6    2    8    6  922    0    7    0]
 [   0   15   16    9   18    2    0  919    5   44]
 [  23    3    5   22    6   18    9    0  882    6]
 [   8    3    4   15   57    9    0   22   14  877]]
{'eval_loss': 0.23178459703922272, 'eval_f1': 0.9282962114422455, 'eval_precision': 0.9285663043775354, 'eval_recall': 0.9285, 'eval_accuracy': 0.9285, 'eval_runtime': 3.5961, 'eval_samples_per_second': 2780.803, 'eval_steps_per_second': 43.659, 'epoch': 10.0}
{'train_runtime': 340.4996, 'train_samples_per_second': 1762.117, 'train_steps_per_second': 27.548, 'train_loss': 0.44155973503584545, 'epoch': 10.0}


TrainOutput(global_step=9380, training_loss=0.44155973503584545, metrics={'train_runtime': 340.4996, 'train_samples_per_second': 1762.117, 'train_steps_per_second': 27.548, 'train_loss': 0.44155973503584545, 'epoch': 10.0})

In [8]:
trainer.evaluate()

  0%|          | 0/157 [00:00<?, ?it/s]

Confusion Matrix:
[[ 944    0    5    0    6    3    7    2   12    1]
 [   0 1119    3    2    0    1    4    0    6    0]
 [   8    9  968   20    2   11    4    8    2    0]
 [   2    8   21  949    0   12    1   10    4    3]
 [   2    5   11    2  897   10   12    7   10   26]
 [   3    3   15   35    2  808    8    1   12    5]
 [   5    2    6    2    8    6  922    0    7    0]
 [   0   15   16    9   18    2    0  919    5   44]
 [  23    3    5   22    6   18    9    0  882    6]
 [   8    3    4   15   57    9    0   22   14  877]]


{'eval_loss': 0.23178459703922272,
 'eval_f1': 0.9282962114422455,
 'eval_precision': 0.9285663043775354,
 'eval_recall': 0.9285,
 'eval_accuracy': 0.9285,
 'eval_runtime': 3.4166,
 'eval_samples_per_second': 2926.901,
 'eval_steps_per_second': 45.952,
 'epoch': 10.0}

In [6]:
import torch
from torch import nn
from torchvision import transforms, datasets
from bitlinear import BitLinear as Linear

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_size = 784
hidden_size = 100
num_classes = 10
learning_rate = 0.001
num_epochs = 5


train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)


class Net(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Net, self).__init__()
        self.fc1 = Linear(input_size, hidden_size)
        self.fc2 = Linear(hidden_size, num_classes)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.log_softmax(self.fc2(x), dim=1)
        return x


# Create the model and optimizer
model = Net(input_size, hidden_size, num_classes)
model = model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)
criterion = nn.NLLLoss()

# Train the model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Forward pass
        outputs = model(images.view(-1, 28 * 28).to(device)).cpu()
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward(retain_graph=False)
        optimizer.step()

        # Print training information
        if (i + 1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

Epoch [1/5], Step [100/938], Loss: 2.1903
Epoch [1/5], Step [200/938], Loss: 2.0577
Epoch [1/5], Step [300/938], Loss: 1.9495
Epoch [1/5], Step [400/938], Loss: 1.8025
Epoch [1/5], Step [500/938], Loss: 1.6161
Epoch [1/5], Step [600/938], Loss: 1.5318
Epoch [1/5], Step [700/938], Loss: 1.4546
Epoch [1/5], Step [800/938], Loss: 1.3805
Epoch [1/5], Step [900/938], Loss: 1.2134
Epoch [2/5], Step [100/938], Loss: 1.2599
Epoch [2/5], Step [200/938], Loss: 1.0751
Epoch [2/5], Step [300/938], Loss: 1.2669
Epoch [2/5], Step [400/938], Loss: 1.0585
Epoch [2/5], Step [500/938], Loss: 1.0870
Epoch [2/5], Step [600/938], Loss: 0.9355
Epoch [2/5], Step [700/938], Loss: 0.9594
Epoch [2/5], Step [800/938], Loss: 0.9366
Epoch [2/5], Step [900/938], Loss: 0.9701
Epoch [3/5], Step [100/938], Loss: 0.9047
Epoch [3/5], Step [200/938], Loss: 0.7651
Epoch [3/5], Step [300/938], Loss: 0.8008
Epoch [3/5], Step [400/938], Loss: 0.7867
Epoch [3/5], Step [500/938], Loss: 0.8615
Epoch [3/5], Step [600/938], Loss:

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument weight in method wrapper_CUDA__native_layer_norm)

In [8]:
# Test the model
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images.view(-1, 28 * 28).to(device)).cpu()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')

Accuracy of the network on the 10000 test images: 89.72%
